In [1]:
import os, sys
import numpy as np


from lace.emulator.nn_architecture import MDNemulator_polyfit
from lace.emulator.nn_emulator import NNEmulator
from lace.emulator.gp_emulator import GPEmulator
from lace.emulator.emulator import P1D_emulator


/data/astro/scratch/lcabayol/anaconda3/envs/DESIenv6/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# our modules
from lace.cosmo import fit_linP
from lace.emulator import poly_p1d
from lace.archive import pnd_archive
from lace.archive import interface_archive

In [3]:
emuparams = ['Delta2_p', 'n_p','mF', 'sigT_Mpc', 'gamma', 'kF_Mpc']

# CREATE TRAINING AND TESTING ARCHIVE

In [4]:
archive = pnd_archive.archivePND(sim_suite="Pedersen21")
archive.get_training_data()
len(archive.training_data)

330

In [5]:
archive_test = pnd_archive.archivePND(sim_suite="Cabayol23", pick_sim='central')
archive_test.get_testing_data()
len(archive_test.testing_data)

11

In [6]:
kMpc = archive_test.testing_data[0]['k_Mpc'][1:43]

# EMULATOR INTERFACE

## There are two possibilities to define the emulator algorithm: 
 - Define emu_algorithm to either GP or NN
 - Pass an emulator_label pointing to the concrete emulator used in a publication. Options are [Pedersen21, Pedersen23, Cabayol23]

## There are two possibilities to define the training archive: 
 - Pass a custom archive 
 - Pass a training_set pointing to the concrete archive used in a publication. Options are [Pedersen21, Cabayol23]
 - If none of them is provided but a emulator label is defined, the emulator will load by default the archive corresponding to the publication
 - If none of them is provided and emulator label is not provided, the emulator crushes
 - If both archive and training_set are provided, the emulator crushes

#### Example 1: Define emu_algorithm and custom archive

In [7]:
emulator = P1D_emulator(archive, emu_algorithm='NN',sims_label='Gadget')

Use custom archive provided by the user
Selected custom emulator


#### Example 2: Pass an emulator label with a custom archive:

In [8]:
emulator = P1D_emulator(archive, emulator_label='Cabayol23')

Use custom archive provided by the user
Selected emulator in Cabayol23
Neural network emulator predicting the optimal P1D fitting coefficients to a 5th degree polynomial. It goes to scales of 4Mpc^{-1} and z<4.5. The parameters passed to the emulator will be overwritten to match these ones


#### Example 3: Pass an emulator label and a training_set label

In [21]:
emulator = P1D_emulator(training_set='Cabayol23', emulator_label='Cabayol23')

Selected archive in Cabayol23
Selected emulator in Cabayol23
Neural network emulator predicting the optimal P1D fitting coefficients to a 5th degree polynomial. It goes to scales of 4Mpc^{-1} and z<4.5. The parameters passed to the emulator will be overwritten to match these ones


####  Example 4: Pass only an emulator label

In [10]:
emulator = P1D_emulator(emulator_label='Cabayol23')

Selected emulator in Cabayol23
Neural network emulator predicting the optimal P1D fitting coefficients to a 5th degree polynomial. It goes to scales of 4Mpc^{-1} and z<4.5. The parameters passed to the emulator will be overwritten to match these ones
Setting the archive to that used in None


#### Example 5: Providing archive and training_set

In [7]:
emulator = P1D_emulator(archive=archive,training_set='Cabayol23')

ValueError: Conflict! Both custom archive and training_set provided

#### Example 6: Not providing anything

In [8]:
emulator = P1D_emulator(emu_algorithm='NN')

Exception: Archive, training_set or emulator_label must be provided

#### Example 8: NN emulator loading a pre-trained model

In [13]:
emulator = P1D_emulator(training_set='Cabayol23', emulator_label='Cabayol23',model_path='NNmodels/NNEmulator_LaCEHC.pt', train=False)






Selected archive in Cabayol23
Selected emulator in Cabayol23
Neural network emulator predicting the optimal P1D fitting coefficients to a 5th degree polynomial. It goes to scales of 4Mpc^{-1} and z<4.5. The parameters passed to the emulator will be overwritten to match these ones
Load pre-trained emulator
Model loaded. No training needed
The loaded model corresponds to the following emulation configuration:
:zmax: 4.5, kmax_Mpc: 4, ndeg: 5, emu_params: ['Delta2_p', 'n_p', 'mF', 'sigT_Mpc', 'gamma', 'kF_Mpc'], drop_sim': None


#### Example 8b: NN emulator loading a pre-trained model. If the arguments provided do not coincide with the especifications of the trained mode, the emulator crushes

In [20]:
emulator = P1D_emulator(emu_algorithm='NN', sims_label='Gadget',training_set='Pedersen21',model_path='NNmodels/NNEmulator_LaCEHC.pt', train=False, kmax_Mpc=5)






Selected archive in Pedersen21


TypeError: exceptions must derive from BaseException

#### Example 9: Gaussian process from Pedersen21 without specifying archive

In [9]:
kMpc = archive_test.testing_data[0]['k_Mpc'][1:33]
emulator = P1D_emulator(emulator_label='Pedersen21')

Selected emulator in Pedersen21
Select emulator used in Pedersen et al. 2021
Gaussian Process emulator predicting the P1D at each k-bin. It goes to scales of 3Mpc^{-1} and z<4.5. The parameters passed to the emulator will be overwritten to match these ones.
Setting the archive to that used in Pedersen21


 /data/astro/scratch/lcabayol/anaconda3/envs/DESIenv6/lib/python3.10/site-packages/paramz/parameterized.py:61: RuntimeWarning:Don't forget to initialize by self.initialize_parameter()!


Training GP on 330 points
GPs optimised in 1.61 seconds


#### Example 10: Gaussian process from Pedersen21 also specifying archive

In [11]:
kMpc = archive_test.testing_data[0]['k_Mpc'][1:33]
emulator = P1D_emulator(emulator_label='Pedersen21', archive=archive)

Use custom archive provided by the user
Selected emulator in Pedersen21
Select emulator used in Pedersen et al. 2021
Gaussian Process emulator predicting the P1D at each k-bin. It goes to scales of 3Mpc^{-1} and z<4.5. The parameters passed to the emulator will be overwritten to match these ones.
Training GP on 330 points
GPs optimised in 2.28 seconds


In [7]:
emulator = P1D_emulator(archive, emu_algorithm='NN',sims_label='Gadget',model_path='NNmodels/NNEmulator_LaCEHC.pt', train=False)

Selected custome emulator
Selected custome training set
Loading emulator using a specific archive
Model loaded. No training needed


In [8]:
p1d = emulator.emulate_p1d_Mpc(archive_test.testing_data[0],kMpc)

### train the emulator

In [ ]:
emulator = P1D_emulator(emulator_label='Cabayol23',nepochs_nn=1)

### Load trained emulator

In [ ]:
emulator = P1D_emulator(emulator_label='Cabayol23', train=False, model_path='NNmodels/NNEmulator_LaCEHC.pt')

In [ ]:
# This is supposed to fail
emulator = P1D_emulator()

In [ ]:
emulator = P1D_emulator(archive=archive, emu_algorithm='NN')

In [ ]:
emulator = P1D_emulator(archive=archive, emu_algorithm='GP')